In [1]:
import laspy as lp
import numpy as np
import scipy
from laspy.file import File
from scipy.spatial.kdtree import KDTree
import torch 
import pickle
import time as time
import matplotlib.pyplot as plt
import scipy.stats
from numpy.random import (normal,uniform)
import matplotlib.cm as cm
import itertools
from matplotlib.patches import Circle
from glob import glob
from os.path import join
import itertools
import inspect

In [2]:
class cluster():
    def __init__(self, points):
        self.points = np.array(points)
        self.roughness = np.nanstd(self.points[:,-1])

    def boundpoints(self):
        x_min = np.min(self.points[:,0])
        x_max = np.max(self.points[:,0])
        y_min = np.min(self.points[:,1])
        y_max = np.max(self.points[:,1])
        return np.array([[x_min,x_max],[y_min,y_max]])


class scan(cluster):
    def __init__(self,filepath):
        start = time.time()
        self.file = File(filepath,mode="r")
        self.scale = self.file.header.scale[0]
        self.offset = self.file.header.offset[0]
        self.tree = KDTree(np.vstack([self.file.x, self.file.y, self.file.z]).transpose())
        self.time = self.file.header.get_date()
        end = time.time() - start
        print("Time Elapsed: {}".format(end))

    def nearNeighbor(self,point,k=1):
        return self.tree.query(point,k=k)
    
    def NNN(self,point,k):
        return self.tree.data[self.tree.query(point,k=k)[1]]
    
    def radialcluster(self,point,radius):
        neighbor = self.tree.data[self.tree.query(point,k=1)[1]]
        points = self.tree.data[self.tree.query_ball_point(neighbor,radius)]
        return np.array(points)

'''
1) Generate a set of clusters in an observer class with a time stamp

2) Generate hypothesis points according to a normal distribution

3) Weight each hypothesis point with F(Hypthesis, Cluster) function

4) Resample 
'''

In [132]:
class Particleset():  
    
    def __init__(self,points=None,center=None,weight=None):
        self.points = points
        self.center = center
        self.weight = weight
        
    def set_weight(self,weight):
        self.weight = weight
        
    def gen_covariance(self):
        if self.points is not None:
            self.cov = np.cov(m=self.points.T)
        

class Observer(Particleset):
    def __init__(self,obs):
        self.observations = obs
        self.observations.sort(key= lambda scan: scan.time)
        #self.particles = None
        self.test = Particleset()
        
    def particle_centroid(self,mean, std, N):
        particles = np.empty((N, 3))
        particles[:, 0] = normal(mean[0],std[0],N) # x pos
        particles[:, 1] = normal(mean[1],std[1],N) # y pos
        particles[:, 2] = normal(mean[2],std[2],N)
        return particles
    
    def particle_clusters(self,particles,observer_index):
        particle_set = []
        for i in range(particles.shape[0]):
            points = self.observations[observer_index].NNN(particles[i,:],k=100)
            cluster = Particleset(points=points,center=particles[i,:])
            cluster.gen_covariance()
            particle_set.append(cluster)
        return particle_set
    
    def gen_particles(self,mean,std,N,obs_indx):
        particles = self.particle_centroid(mean,std,N)
        particle_set = self.particle_clusters(particles,obs_indx)
        return particle_set
        
    def kernel_function(self,test,ref):
        # Evaluate the liklihood of one KDE under another
        mean_test = test.points
        cov_test = test.cov
        mean_ref = ref.points
        cov_ref = ref.cov
        assert mean_test.shape == mean_ref.shape
        gamma = mean_test - mean_ref
        sigma = cov_test + cov_ref
        A = 1/((2*np.pi**(3/2))*(np.linalg.det(sigma)**(.5)))
        B = np.exp((-.5)*gamma@np.linalg.inv(sigma)@gamma.T)
        C = 1/(np.max(mean_test.shape))
        return np.sum(A*B)
    
    def evaluate(self,testset,ref):
        for cluster in testset:
            weight = self.kernel_function(cluster,ref)
            cluster.set_weight(weight)
            
        
    ###################
    '''
    def kernel(self,x):
        return ((1.0/(2.0*np.pi))**3.0)*np.exp(-(np.linalg.norm(x)**2/2.0))
        
    def d_sigma(self,X,Y):
        # X is hypothesis, list of particle sets
        # Y is the reference cluster

        ref = [Y[i,:] for i in range(Y.shape[0])]
        hyp = [X[i,:] for i in range(X.shape[0])]
        mat = []
        
        #if isinstance(X,type(self.test)) :
        for cluster in X:
            points = [cluster.points[i,:] for i in range(cluster.points.shape[0])]
            dists = np.array([np.abs(xy[0] - xy[1]) for xy in itertools.product(points,ref)])
            mat.append(np.mean(dists,axis=0))
        #else:
           # mat = X - Y
        
        mat = np.array([np.abs(xy[0] - xy[1]) for xy in itertools.product(hyp,ref)])
        #mat = np.array(mat)
        mat = mat.T@mat
        try:
            L = np.linalg.cholesky(mat)
            H = np.linalg.inv(L)
            H = H.T@H
            return np.sqrt(mat.T@H@mat)
        except:
            print("Non - Singular \n")
            return 0
    
    def bandwidth(self,X):
        return np.min(self.d_sigma(X,X))
    
    def reach_distance(self,X,Y):
        A = self.d_sigma(X,Y)
        B = self.d_sigma(Y,Y)
        if np.sum(A) > np.sum(B):
            return A
        else:
            return B
        
        
    def evaluate(self,X,Y):
        # X is hypothesis
        # Y is reference
        A = self.reach_distance(X,Y)
        B = self.bandwidth(Y)
        C = (1/(B**3))*self.kernel(A/B)
        return(np.mean(C))
    
    def weight_cluster(self,X,Y):
        # assign a cluster weight given a reference cluster
        # X is hypothesis
        # Y is reference
        for particleset in X:
            weights = self.evaluate(particleset.points,Y)
            particleset.set_weight(weights)
       
    '''
    def resample(self,particle_set):
        weights = np.array([points.weight for points in particle_set])
        cumsum = np.cumsum(weights)
        B = uniform(0,1,np.max(weights.shape))
        indexes = np.squeeze(np.searchsorted(cumsum,B))
        print(indexes)
        print(weights.shape)
        print(cumsum)
        print(B)
        for i, particle in enumerate(particle_set):
                particle.weight = particle_set[indexes[i]].weight

        
    def estimate(self,particle_set):
        positions = [cluster.center for cluster in particle_set]
        weights = [particle.weight for particle in particle_set]
        mean = np.average(positions,axis=0,weights=weights)
        var = np.var(positions,axis=0)
        return mean, np.linalg.norm(var)  #returns the estimated location
    
    def predict(self,point,guess,ref_index,test_index):
        ref_cluster = Particleset(points=self.observations[ref_index].NNN(point,k=100))
        ref_cluster.gen_covariance()
        hypothesis = self.gen_particles(guess,np.array([5,5,1]),50,test_index)
        self.evaluate(hypothesis,ref_cluster)
        self.resample(hypothesis)
        return self.estimate(hypothesis)



In [ ]:
files = glob(join("/home/dunbar/Research/helheim/data/misc/lazfiles","*.laz"))
scans = [scan(file) for file in files[:2]]

In [ ]:
observer = Observer(scans)

In [ ]:
def guess(point):
    dt = np.array([10,10,5])
    point += dt
    return point
start = np.array([534330.24,7361293.11,175])
guess = guess(start)

In [ ]:
estimate = observer.predict(point=start,guess=guess,ref_index=0,test_index=1)

In [107]:
displacement = estimate[0] - start
np.abs(displacement)

array([1.10748976, 0.88610618, 0.07415133])